<h1>CrimeCast: Forecasting Crime Categories</h1>

**Name:** Surya Vikram | **Roll No.:** 22F3002751 | **Student Email:** [22f3002751@ds.study.iitm.ac.in](mailto:22f3002751@ds.study.iitm.ac.in)

---

### **_Objective_**
> To explore the comprehensive dataset and construct predictive models that can forecast the category of crime for each incident.

### **_Dataset_**

> Each entry encapsulates a unique narrative, featuring details such as incident locations, victim demographics, and other key attributes.
The detailed features and their descriptions are as follows:

>- **Location:** Street address of the crime incident.
>- **Cross_Street:** Cross street of the rounded address.
>- **Latitude:** Latitude coordinates of the crime incident.
>- **Longitude:** Longitude coordinates of the crime incident.
>- **Date_Reported:** Date the incident was reported.
>- **Date_Occurred:** Date the incident occurred.
>- **Time_Occurred:** Time the incident occurred in 24-hour military time.
>- **Area_ID:** LAPD's Geographic Area number.
>- **Area_Name:** Name designation of the LAPD Geographic Area.
>- **Reporting_District_no:** Reporting district number.
>- **Part 1-2:** Crime classification.
>- **Modus_Operandi:** Activities associated with the suspect.
>- **Victim_Age:** Age of the victim.
>- **Victim_Sex:** Gender of the victim.
>- **Victim_Descent:** Descent code of the victim.
>- **Premise_Code:** Premise code indicating the location of the crime.
>- **Premise_Description:** Description of the premise code.
>- **Weapon_Used_Code:** Weapon code indicating the type of weapon used.
>- **Weapon_Description:** Description of the weapon code.
>- **Status:** Status of the case.
>- **Status_Description:** Description of the status code.
>- **Crime_Category:** The category of the crime (Target Variable).

### **_Libraries_**
> #### **Data Analysis**
   >- Pandas    
   >- Numpy
   >- Scipy
   
> #### **Data Visualization**
   >- Matplotlib
   >- Seaborn

> #### **Modeling**
   >- Scikit-Learn
   >- XGBoost

# Table of Contents

- 1. [Importing Libraries](#Imports)

- 2. [Loading Data](#Load-Data)

- 3. [Exploratory Data Analysis](#EDA)

    - 3.1 [Location Features](#Location)
    
    - 3.2 [Date-Time Features](#Date-Time)
    
    - 3.3 [Area Features](#Area)
    
    - 3.4 [Crime Features](#Crime)
    
    - 3.5 [Victim Features](#Victim)
    
    - 3.6 [Premise Features](#Premise)
    
    - 3.7 [Weapon Features](#Weapon)
    
    - 3.8 [Status Features](#Status)

- 4. [Preprocessing](#Preprocessing)

- 5. [Model Training and Selection](#Model-Training-Selection)

   - 5.1 [Train-Validation Datasets](#Train-Validation)
    
   - 5.2 [Helper Functions](#Helper)
    
   - 5.3 [Models](#Model)
    
       - 5.3.1 [Logistic Regression](#Logistic-Regression)
      
       - 5.3.2 [K-Nearest Neighbors](#KNN)
      
       - 5.3.3 [Support Vector Classifier](#SVC)
       
       - 5.3.4 [Decision Tree](#DT)
       
       - 5.3.5 [Random Forest - Bagging](#Random-Forest)
       
       - 5.3.6 [XGBoostClassifier - Boosting](#XGBoost)
       
       - 5.3.7 [Multi-Layer Perceptron](#MLP)
    
    - 5.4 [Model Comparison](#Model-Compare)
    
       - 5.4.1 [Comparing Metrics](#Compare-Metrics)
      
       - 5.4.2 [ROC Curves](#ROC-Curve)
      
    - 5.5 [Ensemble](#Ensemble)
    
       - 5.5.1 [Voting Classifier](#Voting)
      
         - 5.5.1.1 [Hard Voting Classifier](#Hard-Voting)
      
         - 5.5.1.2 [Soft Voting Classifier](#Soft-Voting)
      
       - 5.5.2 [Stacking Classifier](#Stacking)
    
    - 5.6 [Model Comparison Part-2](#Model-Compare-2)
    
       - 5.6.1 [ROC Curves](#ROC-Curve-2)
       
       - 5.6.2 [Confusion and Error Matrices](#Confusion-Error-Matrix)

- 6. [Submission](#Submission)

<a name='Imports'></a>
# 1. Importing Libraries

In [ ]:
import os
import re
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import scipy.stats as stats
from scipy.stats import chi2, chi2_contingency
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, log_loss, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedShuffleSplit, cross_validate, validation_curve, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, LabelEncoder, MultiLabelBinarizer

<a name='Load-Data'></a>
# 2. Loading Data

In [ ]:
paths = {}
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename == 'train.csv':
            paths['train'] = os.path.join(dirname, filename)
        elif filename == 'test.csv':
            paths['test'] = os.path.join(dirname, filename)
        else:
            paths['sample'] = os.path.join(dirname, filename)

In [ ]:
train = pd.read_csv(paths['train'])
test = pd.read_csv(paths['test'])
sample = pd.read_csv(paths['sample'])

In [ ]:
# Preview Train Data
train.head()

In [ ]:
# Preview Test Data
test.head()

In [ ]:
# Preview Sample submission
sample.head()

In [ ]:
# Checking shape of datasets
train.shape, test.shape, sample.shape

<a name='EDA'></a>
# 3. Exploratory Data Analysis

In [ ]:
train.info()

In [ ]:
train.sample(10)

In [ ]:
train.isna().sum() / train.shape[0] * 100

In [ ]:
target = train.Crime_Category.copy()

In [ ]:
unique, counts = np.unique(train.Crime_Category, return_counts=True)

percentages = counts / counts.sum() * 100
fig, ax = plt.subplots(figsize=(6, 6))
ax.pie(counts, 
       labels=[f'{label} ({pct:.1f}%)' for label, pct in zip(unique, percentages)], 
       startangle=140)
ax.set_title('Share of Crime Categories', fontsize=14)

plt.show()

In [ ]:
categories = {
    'Location': ['Location', 'Cross_Street', 'Latitude', 'Longitude'],
    'Date_Time': ['Date_Reported', 'Date_Occurred', 'Time_Occurred'],
    'Area': ['Area_ID', 'Area_Name', 'Reporting_District_no'],
    'Crime': ['Part 1-2', 'Modus_Operandi'],
    'Victim': ['Victim_Age', 'Victim_Sex', 'Victim_Descent'],
    'Premise': ['Premise_Code', 'Premise_Description'],
    'Weapon': ['Weapon_Used_Code', 'Weapon_Description'],
    'Status': ['Status', 'Status_Description']
}

<a name='Location'></a>
## 3.1 Location Features

In [ ]:
loc_df = train[categories['Location']].copy()

In [ ]:
loc_df

In [ ]:
loc_df['Location_Type'] = loc_df.Location.apply(lambda val: val.split()[-1])

In [ ]:
loc_df.head()

In [ ]:
loc_df.Location_Type.nunique()

*Due to very high number of unique values we will further trim it down according to their frequency of occurence.*

In [ ]:
loc_df.Location_Type.value_counts()[:10]

In [ ]:
loc_df.drop('Location_Type', axis=1)

Location column seems to have 3 parts concatinated in it which we can extract to reduce the count of unique values. Especially the Type of Location with values such as ST for street, AV for Avenue, RD for road and more.

In [ ]:
valid_street_types = {'ST', 'AV', 'DR', 'BL', 'PL', 'WY', 'LN', 'HY', 'RD', 'BROADWAY'}

def extract_location_components(location):
    parts = location.split()

    street_number = parts[0] if parts[0].isdigit() else np.nan

    street_type = parts[-1].upper() if parts[-1].upper() in valid_street_types else np.nan

    if street_number is not np.nan:
        if street_type is not np.nan:
            street_name = ' '.join(parts[1:-1]).strip()
        else:
            street_name = ' '.join(parts[1:]).strip()
    else:
        if street_type is not np.nan:
            street_name = ' '.join(parts[:-1]).strip()
        else:
            street_name = ' '.join(parts).strip()
    
    street_number = str(street_number) if not pd.isna(street_number) else street_number
    
    return (street_number, street_name.upper() if street_name else street_name, street_type)

In [ ]:
loc_df[['Location_Number', 'Location_Name', 'Location_Type']] = loc_df['Location'].apply(lambda x: pd.Series(extract_location_components(x)))

In [ ]:
loc_df.head()

In [ ]:
for col in loc_df.columns:
    print(col, loc_df[col].nunique())

In [ ]:
loc_df.Cross_Street.value_counts()

Drop Cross_Street due to 1495 unique values and low occurence of even most frequent values.

In [ ]:
loc_df.isna().sum()

In [ ]:
plt.figure(figsize=(20, 8))
sns.set(style="ticks")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

violin_plot_lat = sns.violinplot(y='Latitude', data=loc_df, ax=ax1,
                                 inner=None,
                                 color='green')
for i, artist in enumerate(violin_plot_lat.collections):
    path = artist.get_paths()[0]
    vertices = path.vertices
    ax1.plot([i]*len(vertices), [v[1] for v in vertices], linewidth=2, color='black')

ax1.set_title('Violin Plot of Latitude', fontsize=16)
ax1.set_ylabel('Latitude', fontsize=14)

violin_plot_long = sns.violinplot(y='Longitude', data=loc_df, ax=ax2,
                                  inner=None,
                                  color='green')
for i, artist in enumerate(violin_plot_long.collections):
    path = artist.get_paths()[0]
    vertices = path.vertices
    ax2.plot([i]*len(vertices), [v[1] for v in vertices], linewidth=2, color='black')

ax2.set_title('Violin Plot of Longitude', fontsize=16)
ax2.set_ylabel('Longitude', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
loc_df[['Latitude', 'Longitude']].describe()

**Outliers**

- The minimum value of 0 is clearly an outlier, as it's far from the 25th percentile (34.0092).
- The maximum value of 0 is an outlier, as it's far from the 75th percentile (-118.2744).

**Data Concentration**

- Most latitudes are between 34.0092 and 34.1650 (interquartile range).
- Most longitudes are between -118.4297 and -118.2744 (interquartile range).


In [ ]:
plt.figure(figsize=(6, 4))
sns.heatmap(loc_df[['Latitude', 'Longitude']].corr(), annot=True, cmap='viridis', vmin=-1, vmax=1, cbar=True)
plt.title('Correlation Heatmap for Latitude and Longitude')
plt.show()

In [ ]:
loc_df["Crime_Category"] = train.Crime_Category.copy()
groups = loc_df.groupby("Crime_Category")
plt.figure(figsize=(10, 6))
for name, group in groups:
    plt.scatter(group['Longitude'], group['Latitude'], label=name, s=1)  
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Scatter Plot by Types of Crime')
plt.xlim(-118.1, -118.7)
plt.ylim(33.6, 34.4)
plt.grid(True)
plt.legend(markerscale=4) 
plt.show()
loc_df.drop(columns=["Crime_Category"], axis=1, inplace=True)

In [ ]:
lat_long = loc_df[['Latitude', 'Longitude']].copy()
scaler = StandardScaler()
lat_long_scaled = scaler.fit_transform(lat_long)

sse = []
k_range = range(2, 15)

for k in k_range:
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=0)
    kmeans.fit(lat_long_scaled)
    sse.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(k_range, sse, 'bo-')
plt.xlabel('Number of clusters')
plt.ylabel('Sum of squared distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
k = 5
loc_df['Crime_Category'] = target.copy()

kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
loc_df['Cluster'] = kmeans.fit_predict(lat_long_scaled)

crime_categories = loc_df['Crime_Category'].unique()

crime_distribution_df = pd.DataFrame(index=crime_categories)

for cluster in range(k):
    cluster_data = loc_df[loc_df['Cluster'] == cluster]
    crime_distribution = cluster_data['Crime_Category'].value_counts(normalize=True) * 100
    crime_distribution_df[f'Cluster {cluster}'] = crime_distribution

plt.figure(figsize=(15, 8))

bar_width = 0.1
bar_positions = np.arange(len(crime_categories))

for i in range(k):
    plt.bar(bar_positions + i * bar_width, 
            crime_distribution_df[f'Cluster {i}'].reindex(crime_categories, fill_value=0), 
            width=bar_width, 
            label=f'Cluster {i}')

plt.xlabel('Crime Category')
plt.ylabel('Percentage (%)')
plt.title('Percentage of Each Crime Category in Each Cluster')
plt.xticks(bar_positions + bar_width * (k-1) / 2, crime_categories, rotation=45)
plt.ylim(0, 80)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def preprocess_location(df):

    df[['Location_Number', 'Location_Name', 'Location_Type']] = df['Location'].apply(lambda x: pd.Series(extract_location_components(x)))
    
    df['Location_Type'] = df['Location_Type'].fillna('Unknown')
    
    df = df.drop(columns=['Location', 'Cross_Street', 'Location_Name', 'Latitude', 'Longitude', 'Location_Number'])
    
    return df

In [ ]:
preprocessed_loc_df = preprocess_location(train[categories['Location']].copy())
preprocessed_loc_df.head()

In [ ]:
preprocessed_loc_df.Location_Type.value_counts()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='Location_Type', data=preprocessed_loc_df)
plt.xlabel('Location Types')
plt.ylabel('Count')
plt.xticks(rotation=0)  
plt.show()

<a name='Date-Time'></a>
## 3.2 Date-Time Features

In [ ]:
datetime_df = train[categories['Date_Time']].copy()

In [ ]:
datetime_df

In [ ]:
def preprocess_datetime(df):

    df['Date_Reported'] = pd.to_datetime(df['Date_Reported'], format='%m/%d/%Y %I:%M:%S %p')
    df['Date_Occurred'] = pd.to_datetime(df['Date_Occurred'], format='%m/%d/%Y %I:%M:%S %p')
    df['Time_Occurred'] = df['Time_Occurred'].astype(int).astype(str).str.zfill(4)  
    df['Time_Occurred'] = pd.to_datetime(df['Time_Occurred'], format='%H%M').dt.time
    
    df['Occurred_Month'] = df['Date_Occurred'].dt.month

    df['Occurred_Hour'] = df['Time_Occurred'].apply(lambda x: x.hour)

    df['Is_Weekend'] = (df['Date_Occurred'].dt.dayofweek >= 5).astype(int)

    la_holidays_2020 = ['2020-01-01', '2020-01-20', '2020-02-17', '2020-05-25', '2020-07-04',
                        '2020-09-07', '2020-10-12', '2020-11-11', '2020-11-26', '2020-12-25']
    
    df['Is_Holiday'] = df['Date_Occurred'].dt.date.astype('str').isin(la_holidays_2020).astype(int)

    df['Reporting_Delay'] = (df['Date_Reported'] - df['Date_Occurred']).dt.days

    df = df.drop(['Date_Reported', 'Date_Occurred', 'Time_Occurred'], axis=1)

    return df

In [ ]:
preprocessed_datetime_df = preprocess_datetime(train[categories['Date_Time']].copy())
preprocessed_datetime_df.head()

In [ ]:
plt.figure(figsize=(6, 4))
sns.heatmap(preprocessed_datetime_df.corr(), annot=True, cmap='magma', vmin=-1, vmax=1, cbar=True)
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(x='Occurred_Hour', data=preprocessed_datetime_df)
plt.xlabel('Occured Hour of Crime')
plt.ylabel('Count')
plt.xticks(rotation=90)  
plt.show()

It follows a near-smooth bell curve with most crimes centered around evening.

<a name='Area'></a>
## 3.3 Area Features

In [ ]:
area_df = train[categories['Area']].copy()

In [ ]:
area_df

In [ ]:
area_df.isna().sum()

In [ ]:
area_df.Area_Name.value_counts()

In [ ]:
inconsistencies = (area_df.groupby('Area_ID')['Area_Name'].nunique() != 1).sum()

if inconsistencies == 0:
    print("No inconsistencies between Area_ID & Area_Name. Proceed to drop Area_Name Column as Area_ID works both as categoric and numeric feature.")
else:
    print(f"{inconsistencies} inconsistencies between Area_ID & Area_Name")

In [ ]:
area_df.Area_ID.nunique()

In [ ]:
area_df.Reporting_District_no.value_counts()

*Drop Reporting_District_no due to large number of unique values and less occurence of even most frequent values.*

In [ ]:
def preprocess_area(df):
    df = df.drop(['Area_Name', 'Reporting_District_no'], axis=1)
    return df

<a name='Crime'></a>
## 3.4 Crime Features

In [ ]:
combined = pd.concat([train.drop(columns=['Crime_Category']), test], ignore_index=True)
def extract_unique_modus_operandi(modus_operandi_series):
    modus_operandi_series = modus_operandi_series.dropna()
    modus_operandi_list = modus_operandi_series.str.split().tolist()
    flattened_list = [item for sublist in modus_operandi_list for item in sublist]
    unique_modus_operandi = set(flattened_list)
    return list(unique_modus_operandi)

In [ ]:
len(extract_unique_modus_operandi(combined.Modus_Operandi))

In [ ]:
crime_df = train[categories['Crime']].copy()
crime_df.head()

In [ ]:
grouped = train.groupby(['Crime_Category', 'Part 1-2']).size().unstack()
plt.figure(figsize=(6, 6))
sns.heatmap(grouped, cmap='YlGnBu', annot=True)  
plt.xlabel('Part 1-2', fontsize=16)
plt.ylabel('Crime Category', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
crime_df.isna().sum() / crime_df.shape[0] * 100

**Due to high percentage of null values in Modus_Operandi imputation could introduce bias and hence introduing a new value seems viable.**

In [ ]:
def preprocess_crime(df):
    df['Modus_Operandi_Unknown'] = df['Modus_Operandi'].isna().astype(int)   
    df['Modus_Operandi'] = df['Modus_Operandi'].fillna('')   
    df['Modus_Operandi'] = df['Modus_Operandi'].str.split()   
    return df

<a name='Victim'></a>
## 3.5 Victim Features

In [ ]:
victim_df = train[categories['Victim']].copy()

In [ ]:
victim_df

In [ ]:
bins = list(range(0, 101, 10))
plt.figure(figsize=(10, 6))
sns.histplot(victim_df['Victim_Age'], bins=bins, kde=False, color='blue', edgecolor='black')
plt.xlabel('Victim Age')
plt.ylabel('Frequency')
plt.title('Histogram of Victim Age')
plt.grid(True)
plt.show()

Children in the age range 0 to 10 are the Most Frequent Victims.

In [ ]:
victims_0_to_10 = victim_df[(victim_df['Victim_Age'] >= 0) & (victim_df['Victim_Age'] <= 10)]

plt.figure(figsize=(10, 6))
sns.histplot(victims_0_to_10['Victim_Age'], bins=11, kde=False, color='blue', edgecolor='black')
plt.xlabel('Victim Age (0-10)')
plt.ylabel('Frequency')
plt.title('Histogram of Victim Age (0-10)')
plt.grid(True)

plt.show()

In [ ]:
filtered_df = train[train['Victim_Age'] == 0]
crime_counts = filtered_df['Crime_Category'].value_counts()
crime_percentage = (crime_counts / filtered_df.shape[0]) * 100
print(crime_percentage)

### Interpretations and Actions
1. **Crimes are committed against children to take their property**: The data shows a significant proportion of crimes where the victim's age is recorded as 0 are property-related crimes. This might indicate that children are being targeted for their possessions. 
   - **Action**: Leave the age as 0 in such cases to reflect the true nature of the data.
   
2. **It is used as a placeholder where the victim's identity is not known**: The presence of many entries with Victim_Age as 0 suggests that it could be a default value used when the actual age of the victim is not known or not recorded.
   - **Action**: One-hot encode these cases to distinguish them from other records. This avoids the introduction of bias that can occur if ages are imputed.

3. **Potentially represent cases where the victim is not a person or the owner is not a single person (e.g., a franchise, a company, a business, a public property)**: In some instances, Victim_Age being 0 may indicate that the crime was committed against an entity such as a business or public property, where the concept of a victim's age does not apply.
   - **Action**: One-hot encode these cases as well to handle them separately and avoid introducing bias through imputation.


In [ ]:
train[train['Victim_Age'] == 0].sample(10).loc[:, ['Victim_Age', 'Victim_Sex', 'Victim_Descent', 'Crime_Category']]

Based on the above sample, Age 0 most likely corresponds to newborns or unknowns, as Age 0 appears in combination with different sexes and descents.

In [ ]:
for col in victim_df.columns:
    print(col)
    print(victim_df[col].unique())

In [ ]:
victim_df.isna().sum() / crime_df.shape[0] * 100

**Due to high percentage of null values in both Victim_Sex and Victim_Descent imputation could introduce bias and hence introduing a new value inplace of null seems viable.**

In [ ]:
train[victim_df.Victim_Age < 0].iloc[:, 10:]

**Victim is very less likely to be less than 20 years of Age due to the nature of Property Crimes and Frauds. Imputation with mean could work.**

In [ ]:
victim_df.Victim_Age.describe()

In [ ]:
def preprocess_victim(df):    
    df.loc[df['Victim_Age'] < 0, 'Victim_Age'] = 30.0
    df['Victim_Sex'] = df['Victim_Sex'].fillna('Unknown')
    df['Victim_Descent'] = df['Victim_Descent'].fillna('Unknown')    
    return df

<a name='Premise'></a>
## 3.6 Premise Features

In [ ]:
premise_df = train[categories['Premise']].copy()

In [ ]:
premise_df

In [ ]:
inconsistencies = (premise_df.groupby('Premise_Code')['Premise_Description'].nunique() != 1).sum()

if inconsistencies == 0:
    print("No inconsistencies between Premise_Code & Premise_Description]. Proceed to drop Premise_Description.")
else:
    print(f"{inconsistencies} inconsistencies between Premise_Code & Premise_Description")

### 3.6.1 Hypothesis: Premise_Code and Premise_Description are highly dependent features

In [ ]:
# Contingency Table for Premise_Code and Premise_Description
premise_contingency_table = premise_df.groupby('Premise_Code')['Premise_Description'].value_counts().unstack().replace(np.nan, 0)

Perform $\chi^2$ Test of Independence for Premise Contingency Table

- $\text{H}_0$: Premise_Code and Premise_Description are independent
- $\text{H}_{\text{A}}$: Premise_Code and Premise_Description are dependent

In [ ]:
# Perform Chi-square Test of Independence
chi2_comp, p_value, dof, expected_ct = chi2_contingency(premise_contingency_table)

alpha = 0.001
chi2_tab = chi2.ppf(1 - alpha, dof)

print(f'Computed Chi-square:  {chi2_comp:.4f}')
print(f'Tabular Chi-square :  {chi2_tab:.4f}\n')

print(f'p-Value            :  {p_value}')
print(f'Significance Level :  {alpha}')

In [ ]:
# Reject or Fail to reject Null Hypothesis
if chi2_comp > chi2_tab and p_value < alpha:
    print('Reject H_0 and conclude Premise_Code and Premise_Description are dependent')
else:
    print('Fail to reject H_0 and conclude Premise_Code and Premise_Description are independent')

*Since we are 99.9% confident about the dependence of Premise_Code and Premise_Description. Drop Premise_Description to reduce multicollinearity.*

In [ ]:
def preprocess_premise(df):
    df = df.drop(columns=['Premise_Description']) 
    return df

<a name='Weapon'></a>
## 3.7 Weapon Features

In [ ]:
weapon_df = train[categories['Weapon']].copy()
weapon_df.head()

In [ ]:
weapon_df.Weapon_Description.nunique()

In [ ]:
weapon_df.Weapon_Description.value_counts()[:5]

The data reveals that "STRONG-ARM" crimes, involving hands, fists, feet, or other forms of bodily force, are overwhelmingly more frequent. This high incidence of strong-arm crimes provides valuable insights into the broader patterns of criminal activity. Specifically, this trend sheds light on the prevalence of property crimes within the dataset.

In [ ]:
weapon_df.isna().sum() / weapon_df.shape[0] * 100

Despite having a large number of null values, the type of weapon strongly influences the type of crime. Dropping this variable might reduce the model's performance; therefore, missing values are imputed with 'Unknown'.

In [ ]:
inconsistencies = (weapon_df.groupby('Weapon_Used_Code')['Weapon_Description'].nunique() != 1).sum()

if inconsistencies == 0:
    print("No inconsistencies between Weapon_Used_Code & Weapon_Description. Proceed to drop Weapon_Description.")
else:
    print(f"{inconsistencies} inconsistencies between Weapon_Used_Code & Weapon_Description")

In [ ]:
def preprocess_weapon(df):    
    df['Weapon_Used_Code'] = df['Weapon_Used_Code'].fillna('Unknown')
    df['Weapon_Used_Code'] = df['Weapon_Used_Code'].astype(str)
    df = df.drop(columns=['Weapon_Description']) 
    return df

<a name='Status'></a>
## 3.8 Status Features

In [ ]:
status_df = train[categories['Status']].copy()
status_df.head()

In [ ]:
status_df.isna().sum() / status_df.shape[0] * 100

In [ ]:
status_df.value_counts()

Status seems to be the abbreviation of Status_Description

In [ ]:
inconsistencies = (status_df.groupby('Status')['Status_Description'].nunique() != 1).sum()

if inconsistencies == 0:
    print("No inconsistencies between Status & Status_Description. Proceed to drop one as both are categorical type.")
else:
    print(f"{inconsistencies} inconsistencies between Status & Status_Description")

In [ ]:
def preprocess_status(df):
    df = df.drop(['Status_Description'], axis=1)
    return df

<a name='Preprocessing'></a>
# 4. Preprocessing

In [ ]:
class DateTimeTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return preprocess_datetime(X)
    
    def get_feature_names_out(self, input_features=None):
        return ['Occurred_Month', 'Occurred_Hour', 'Is_Weekend', 'Is_Holiday', 'Reporting_Delay']

    
class CrimeTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return preprocess_crime(X)
    
    def get_feature_names_out(self, input_features=None):
        return ['Part 1-2', 'Modus_Operandi', 'Modus_Operandi_Unknown']

    
class LocationTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return preprocess_location(X)
    
    def get_feature_names_out(self, input_features=None):
        return ['Location_Type']
    

class VictimTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return preprocess_victim(X)
    
    def get_feature_names_out(self, input_features=None):
        return ['Victim_Age', 'Victim_Sex', 'Victim_Descent']

    
class WeaponTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return preprocess_weapon(X)
    
    def get_feature_names_out(self, input_features=None):
        return ['Weapon_Used_Code']  
    

class StatusTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return preprocess_status(X)
    
    def get_feature_names_out(self, input_features=None):
        return ['Status']

    
class AreaTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return preprocess_area(X)
    
    def get_feature_names_out(self, input_features=None):
        return ['Area_ID']
    

class PremiseTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return preprocess_premise(X)
    
    def get_feature_names_out(self, input_features=None):
        return ['Premise_Code']
    
    
class ModusOperandiBinarizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.mlb = MultiLabelBinarizer()
    
    def fit(self, X, y=None):
        self.mlb.fit(X)
        return self
    
    def transform(self, X):
        return self.mlb.transform(X)
    
    def fit_transform(self, X, y=None):
        return self.mlb.fit_transform(X)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('datetime', DateTimeTransformer(), categories['Date_Time']),
        ('crime', CrimeTransformer(), categories['Crime']),
        ('location', LocationTransformer(), categories['Location']),
        ('victim', VictimTransformer(), categories['Victim']),
        ('weapon', WeaponTransformer(), categories['Weapon']),
        ('status', StatusTransformer(), categories['Status']),
        ('area', AreaTransformer(), categories['Area']),
        ('premise', PremiseTransformer(), categories['Premise'])
    ],
    remainder='passthrough',
    n_jobs=-1,  
    verbose_feature_names_out=False
)

In [ ]:
preprocessed_train = preprocessor.fit_transform(train.drop(columns=['Crime_Category']))
preprocessed_train_df = pd.DataFrame(preprocessed_train, columns=preprocessor.get_feature_names_out())

preprocessed_test = preprocessor.transform(test)
preprocessed_test_df = pd.DataFrame(preprocessed_test, columns=preprocessor.get_feature_names_out())

In [ ]:
preprocessed_train_df.info()

In [ ]:
label_encoder = LabelEncoder()
target_encoded = label_encoder.fit_transform(target)

In [ ]:
# Heatmap of Correlation among finalized numerical features and target
columns_list = [
    'Occurred_Month',
    'Occurred_Hour',
    'Is_Weekend',
    'Is_Holiday',
    'Reporting_Delay',
    'Part 1-2',
    'Victim_Age',
    'Area_ID',
    'Premise_Code'
]

num_corr_df = preprocessed_train_df[columns_list].copy()
num_corr_df['Crime_Category'] = target_encoded
plt.figure(figsize=(12, 10))
sns.heatmap(num_corr_df.corr(), annot=True, cmap='viridis', vmin=-1, vmax=1, cbar=True)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
encoder = ColumnTransformer(
    transformers=[
        ('mlb', ModusOperandiBinarizer(), 'Modus_Operandi'),
        ('ohe', OneHotEncoder(handle_unknown='ignore'), ['Premise_Code', 'Area_ID', 'Location_Type', 'Victim_Sex', 'Victim_Descent', 'Weapon_Used_Code', 'Status'])  
    ],
    remainder='passthrough'
)

<a name='Model-Training-Selection'></a>
# 5. Model Training and Selection

<a name='Train-Validation'></a>
## 5.1 Train-Validation Datasets

Splitting preprocessed train dataframe to training and validation sets in a stratified manner in the ratio 80:20 respectively

In [ ]:
# Splitting in stratified manner
X_train, X_val, y_train, y_val = train_test_split(preprocessed_train_df, target_encoded, test_size=0.2, stratify=target_encoded, random_state=42)

In [ ]:
# Sanity Check
assert X_train.shape[0] == y_train.shape[0]
assert X_val.shape[0] == y_val.shape[0]
assert X_train.shape[1] == X_val.shape[1]

X_train.shape, y_train.shape, X_val.shape, y_val.shape

In [ ]:
# Stratified CV for Cross Validation throughout the notebook
cv = StratifiedShuffleSplit(n_splits=2, test_size=0.2, random_state=42)

<a name='Helper'></a>
## 5.2 Helper Functions

In [ ]:
# Helper function for validation against multiple metrics
def multi_metric_validation_curve(model, X, y, param_name, param_range, metrics, cv):
    if isinstance(metrics, str):
        metrics = [metrics]

    train_scores = {metric: [] for metric in metrics}
    test_scores = {metric: [] for metric in metrics}

    for param_val in param_range:
        # Use set_params with the full parameter name for the pipeline
        param_full_name = f'classifier__{param_name}'
        cv_results = cross_validate(model.set_params(**{param_full_name: param_val}), X, y,
                                    scoring=metrics,
                                    cv=cv, return_train_score=True, n_jobs=-1)

        for metric in metrics:
            train_scores[metric].append(cv_results[f'train_{metric}'])
            test_scores[metric].append(cv_results[f'test_{metric}'])

    for metric in metrics:
        train_scores[metric] = np.array(train_scores[metric])
        test_scores[metric] = np.array(test_scores[metric])

    return train_scores, test_scores

In [ ]:
# Helper function to plot Validation Curves
def plot_validation_curve(train_scores, test_scores, model_name, param_name, param_range, metric_name, logx=False, zoom_window=None):
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(11, 5))

    # Zoomed Out Plot
    ax[0].set_title(f'Validation Curve for {model_name} for {param_name}')
    ax[0].set_xlabel(param_name)
    ax[0].set_ylabel(metric_name)
    ax[0].set_ylim(0, 1)

    if logx:
        ax[0].semilogx(param_range, train_scores_mean, label='Training', color='darkorange')
    else:
        ax[0].plot(param_range, train_scores_mean, label='Training', color='darkorange')

    ax[0].fill_between(
        param_range,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.2,
        color="darkorange"
    )

    if logx:
        ax[0].semilogx(param_range, test_scores_mean, label='Testing', color='blue')
    else:
        ax[0].plot(param_range, test_scores_mean, label='Testing', color='blue')

    ax[0].fill_between(
        param_range,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.2,
        color="blue"
    )

    ax[0].legend(loc="best")

    # Zoomed In Plot
    ax[1].set_title(f'Validation Curve for {model_name} for {param_name} (Zoomed In)')
    ax[1].set_xlabel(param_name)
    ax[1].set_ylabel(metric_name)
    if zoom_window:
        ax[1].set_ylim(zoom_window)
    else:
        ax[1].set_ylim(min(train_scores.min(), test_scores.min()) - 0.05, max(train_scores.max(), test_scores.max()) + 0.05)

    if logx:
        ax[1].semilogx(param_range, train_scores_mean, label='Training', color='darkorange')
    else:
        ax[1].plot(param_range, train_scores_mean, label='Training', color='darkorange')

    ax[1].fill_between(
        param_range,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.2,
        color="darkorange"
    )

    if logx:
        ax[1].semilogx(param_range, test_scores_mean, label='Testing', color='blue')
    else:
        ax[1].plot(param_range, test_scores_mean, label='Testing', color='blue')

    ax[1].fill_between(
        param_range,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.2,
        color="blue"
    )

    ax[1].legend(loc="best")
    
    plt.savefig('graph')
    plt.show()

In [ ]:
# Dataframe to store metrics of tuned Models for comparison
results = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision Weighted', 'Recall Weighted', 'F1 Weighted', 'ROC AUC OVR Weighted'])

In [ ]:
# Helper function to compute metrics for comparison and update in results dataframe
def compute_comparison_metrics(model_name, model):
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)

    y_train_pred_proba = model.predict_proba(X_train)
    y_val_pred_proba = model.predict_proba(X_val)

    accuracy = [[accuracy_score(y_train, y_train_pred), accuracy_score(y_val, y_val_pred)]]

    prf = [[metric(y_train, y_train_pred, average='weighted'), metric(y_val, y_val_pred, average='weighted')] \
           for metric in [precision_score, recall_score, f1_score]]

    roc_auc = [[roc_auc_score(y_train, y_train_pred_proba, average='weighted', multi_class='ovr'),
                roc_auc_score(y_val, y_val_pred_proba, average='weighted', multi_class='ovr')]]

    results.loc[len(results.index)] = [model_name] + accuracy + prf + roc_auc

<a name='Model'></a>
## 5.3 Models

<a name='Logistic-Regression'></a>
### 5.3.1 Logistic Regression

**Code:**
```python
    model = LogisticRegression(max_iter=1, warm_start=True, solver='saga', random_state=42, n_jobs=-1)

    pipe = Pipeline(steps=[
        ('encoder', encoder),
        ('scaler', StandardScaler()),
        ('classifier', model)
    ])

    iterations = []
    train_loss_values = []
    test_loss_values = []

    for i in range(1, 200):
        pipe.fit(X_train, y_train)

        y_train_pred_proba = pipe.predict_proba(X_train)
        y_test_pred_proba = pipe.predict_proba(X_val)

        train_loss = log_loss(y_train, y_train_pred_proba)
        test_loss = log_loss(y_val, y_test_pred_proba)

        iterations.append(i)
        train_loss_values.append(train_loss)
        test_loss_values.append(test_loss)

    plt.figure(figsize=(10, 6))
    plt.plot(iterations, train_loss_values, label='Train Loss')
    plt.plot(iterations, test_loss_values, label='Test Loss')
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.title('Iteration vs. Loss for Logistic Regression')
    plt.legend()
    plt.grid(True)
    plt.show()
```

**Output:**

<img src='https://drive.google.com/thumbnail?id=1N1qN_FCoj01el87UW4_aEnkooJbdTsz0&sz=w1000' width='1000'>

#### Scaling has a significant impact on convergence speed. Without using StandardScaler, convergence took around 4000 iterations, but now it is approximately 100 iterations.

**Code:**
```python
    # Tuning Logistic Regression for different regularization strengths
    model = LogisticRegression(max_iter=100, random_state=42, n_jobs=-1, solver='saga')

    pipe = Pipeline(steps=[
        ('encoder', encoder),
        ('scaler', StandardScaler()),
        ('classifier', model)
    ])

    C_range = np.logspace(-3, 1, 5)

    train_scores, test_scores = multi_metric_validation_curve(pipe, X_train, y_train,
                                                              param_name='C', param_range=C_range,
                                                                  metrics=['neg_log_loss', 'roc_auc_ovr_weighted'], cv=cv)
    # Plot Validation Curve for Log Loss
    plot_validation_curve(-train_scores['neg_log_loss'], -test_scores['neg_log_loss'],
                          'Logistic Regression', 'C', C_range,
                          'Cross-Entropy Loss', logx=True, zoom_window=(0.0, 0.6))
```

**Output:**

<img src='https://drive.google.com/thumbnail?id=1u3xEbwOV82-YO3fBXbE4kIFQpK4D2oue&sz=w1000' width='1000'>

---

**Code:**
```python
    # Plot Validation Curve for ROC AUC
    plot_validation_curve(train_scores['roc_auc_ovr_weighted'], test_scores['roc_auc_ovr_weighted'],
                          'Logistic Regression', 'C', C_range,
                          'ROC AUC OVR Weighted', logx=True, zoom_window=(0.9, 1.0))
```

**Output:**

<img src='https://drive.google.com/thumbnail?id=1gDlGTFO8mOAdyDHDe7nEP35MUCo5QjDP&sz=w1000' width='1000'>

From the graphs above, C=1 seems most appropriate for both Cross-Entropy Loss and ROC AUC.

In [ ]:
# Tuned Logistic Regression
lr = LogisticRegression(C=1, max_iter=150, random_state=42, n_jobs=-1, solver='saga')
lr_pipe = Pipeline(steps=[
    ('encoder', encoder),
    ('scaler', StandardScaler()),
    ('classifier', lr)
])
lr_pipe.fit(X_train, y_train)

In [ ]:
# Compute metrics for comparison for Logistic Regression
compute_comparison_metrics('Logistic Regression', lr_pipe)
results

<a name='KNN'></a>
### 5.3.2 K-Nearest Neighbors

#### Hyperparameter Tuning KNN
**Code:**
```python
    knn = KNeighborsClassifier(n_jobs=-1)
    pipe = Pipeline(steps=[
        ('encoder', encoder),
        ('feature_selection', SelectKBest(score_func=f_classif)),
        ('scaler', StandardScaler()),
        ('classifier', knn)
    ])
    param_grid = {
        'feature_selection__k': range(10, 61, 10),
        'classifier__n_neighbors': range(15, 28, 3),
        'classifier__weights': ['uniform', 'distance'],
        'classifier__metric': ['euclidean', 'manhattan', 'cosine']
    }
    grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    grid_search.best_params_
```

**Output:**
```
    {'classifier__metric': 'manhattan',
     'classifier__n_neighbors': 24,
     'classifier__weights': 'distance',
     'feature_selection__k': 30}
     
```

In [ ]:
# Tuned KNN
best_params = {'metric': 'manhattan', 'n_neighbors': 24, 'weights': 'distance'}
knn = KNeighborsClassifier(**best_params, n_jobs=-1)
knn_pipe = Pipeline(steps=[
    ('encoder', encoder),
    ('feature_selection', SelectKBest(score_func=f_classif, k=30)),
    ('scaler', StandardScaler()),
    ('classifier', knn)
])
knn_pipe.fit(X_train, y_train)

In [ ]:
# Compute metrics for comparison for KNN
compute_comparison_metrics('K Nearest Neighbors', knn_pipe)
results

<a name='SVC'></a>
### 5.3.3 Support Vector Classifier

#### Hyperparameter Tuning SVC
**Code:**
```python
    svc = SVC(random_state=42)
    pipe = Pipeline(steps=[
        ('encoder', encoder),
        ('classifier', svc)
    ])
    param_grid = {
        'classifier__C': np.logspace(-3, 1, 5),
        'classifier__kernel': ['linear', 'rbf', 'poly', 'sigmoid']
    }
    grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=cv, n_jobs=-1, refit=False)
    grid_search.fit(X_train, y_train)
    grid_search.best_params_
    
```

**Output:**
```
    {'classifier__C': 0.1,
     'classifier__kernel': 'linear'}
     
```

In [ ]:
# Tuned SVC
best_params = {'C': 0.1, 'kernel': 'linear'}
svc = SVC(**best_params, probability=True, random_state=42)
svc_pipe = Pipeline(steps=[
    ('encoder', encoder),
    ('classifier', svc)
])
svc_pipe.fit(X_train, y_train)

In [ ]:
# Compute metrics for comparison for SVC
compute_comparison_metrics('SVC', svc_pipe)
results

<a name='DT'></a>
### 5.3.4 Decision Tree

In [ ]:
# Training a fully grown Decision Tree
dt = DecisionTreeClassifier(random_state=42)
pipe = Pipeline(steps=[
    ('encoder', encoder),
    ('classifier', dt)
])
pipe.fit(X_train, y_train)

In [ ]:
# Checking dimensions of a fully grown tree
dt.get_depth(), dt.get_n_leaves()

In [ ]:
# Plot the Decision Tree
plt.figure(figsize=(15, 4))
plot_tree(dt, max_depth=2, filled=True)
plt.show()

In [ ]:
print(f"Training accuracy: {pipe.score(X_train, y_train):.2f}")
print(f"Testing accuracy: {pipe.score(X_val, y_val):.2f}")

A Decison Tree with long 'branches', depth of 46, and 775 leaf nodes for just 6 classes seems excessive and very prone to overfitting as it may try to fit the noise. Thus, the tree must be pruned. In this notebook, the Decision Tree will be **post-pruned**.

#### **Post-pruning the Decision Tree with Minimal Cost-Complexity Pruning**

Pruning is done to prevent the tree from overfitting. The Cost-Complexity measure $R_{\alpha}(T)$ for a given Tree $T$ and a complexity parameter $\alpha \geq 0$ is

$$
R_{\alpha}(T) = R(T) + \alpha|\tilde{T}|
$$

where $|\tilde{T}|$ is number of terminal nodes and $R(T)$ is the total weighted sample impurity of the terminal nodes. Minimal Cost-Complexity Pruning tries to find the subtree of $T$ that minimizes $R_{\alpha}(T)$.

In [ ]:
# Retrieve the encoder and classifier from the pipeline
encoder = pipe.named_steps['encoder']
classifier = pipe.named_steps['classifier']

# Transform X_train using the encoder
X_train_transformed = encoder.transform(X_train)

# Capture impurities for various ccp_alphas
path = classifier.cost_complexity_pruning_path(X_train_transformed, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
# Plot the effect of CCP alpha on the Impurity
fig, ax = plt.subplots()
ax.plot(ccp_alphas[:-1], impurities[:-1], drawstyle="steps-post")
ax.set_xlabel("Effective Alpha")
ax.set_ylabel("Total Impurity of Leaves")
ax.vlines(x=0.001, ymin=0, ymax=0.1, color='black', linestyles='dashed')
ax.hlines(y=0.1, xmin=0, xmax=0.001, color='black', linestyles='dashed')
ax.set_xlim(0, 0.004)
ax.set_ylim(0, 0.2)
ax.set_title("Total Impurity vs Effective Alpha for Training Set")
plt.show()

print('\nElbow at Alpha~=0.001')

#### Hyperparameter Tuning Decision Tree
**Code:**
```python
    param_grid = {
        'classifier__ccp_alpha': [0.0005, 0.001, 0.002, 0.003],
        'classifier__max_depth': [20, 30, 40, 46],  
        'classifier__min_samples_split': [2, 5, 10, 15],
        'classifier__min_samples_leaf': [2, 4, 7, 10],
    }
    model = DecisionTreeClassifier(random_state=42)
    pipe = Pipeline(steps=[
        ('encoder', encoder),
        ('classifier', model)
    ])
    grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=cv, n_jobs=-1, refit=False)
    grid_search.fit(X_train, y_train)
    grid_search.best_params_
    
```

**Output:**
```
    {'classifier__ccp_alpha': 0.0005,
     'classifier__max_depth': 20,
     'classifier__min_samples_leaf': 2,
     'classifier__min_samples_split': 2}
     
```

In [ ]:
# Tuned DecisionTreeClassifier
dt = DecisionTreeClassifier(ccp_alpha=0.0005, max_depth=20, min_samples_leaf=2, min_samples_split=2, random_state=42)
dt_pipe = Pipeline(steps=[
    ('encoder', encoder),
    ('classifier', dt)
])
dt_pipe.fit(X_train, y_train)

In [ ]:
# Compute metrics for comparison for Decision Tree Classifier
compute_comparison_metrics('Decision Tree', dt_pipe)
results

<a name='Random-Forest'></a>
### 5.3.5 Random Forest - Bagging

#### Hyperparameter Tuning max_depth to avoid overfitting
**Code:**
```python
    rf = RandomForestClassifier(n_estimators=100, bootstrap=True, max_features=None, n_jobs=-1, random_state=42)
    pipe = Pipeline(steps=[
        ('encoder', encoder),  
        ('classifier', rf)
    ])

    max_depth_range = range(9, 25, 3)

    train_scores, test_scores = multi_metric_validation_curve(pipe, X_train, y_train,
                                                              param_name='max_depth', param_range=max_depth_range,
                                                              metrics=['neg_log_loss', 'roc_auc_ovr_weighted'],
                                                              cv=cv)
    # Plot Validation Curve for Log Loss
    plot_validation_curve(-train_scores['neg_log_loss'], -test_scores['neg_log_loss'],
                          'RF', 'No. of Trees', max_depth_range,
                          'Cross-Entropy Loss')
```

**Output:**

<img src='https://drive.google.com/thumbnail?id=1v3erzGbpoITmPCrUjJGZrc53qTm2eabB&sz=w1000' width='1000'>

---

**Code:**
```python
    # Plot Validation Curve for ROC AUC
    plot_validation_curve(train_scores['roc_auc_ovr_weighted'], test_scores['roc_auc_ovr_weighted'],
                          'RF', 'No. of Trees', max_depth_range,
                          'ROC AUC OVR Weighted')
```

**Output:**

<img src='https://drive.google.com/thumbnail?id=1WMJ_uqvxfzfdg_sHVQ_H78NqHIAKO_eU&sz=w1000' width='1000'>

Cross-Entropy Loss for test set starts increasing post-12 max_depth whereas training loss keeps decreasing, yet ROC AUC is maximum around 16 max_depth. Thus, a value in between 12 and 18 (max_depth=16) is considered.

In [ ]:
rf = RandomForestClassifier(n_estimators=100, max_depth=16, max_features=None, bootstrap=True, random_state=42)
rf_pipe = Pipeline(steps=[
    ('encoder', encoder),  
    ('classifier', rf)
])
rf_pipe.fit(X_train, y_train)

In [ ]:
# Compute metrics for comparison for Random Forest
compute_comparison_metrics('Random Forest', rf_pipe)
results

<a name='XGBoost'></a>
### 5.3.6 XGBoostClassifier - Boosting

#### Hyperparameter Tuning XGBoost using RandomizedSearchCV
**Code:**
```python
    xgb = XGBClassifier(random_state=42)

    pipe = Pipeline(steps=[
        ('encoder', encoder),
        ('classifier', xgb)
    ])

    param_dist = {
        'classifier__n_estimators': stats.randint(100, 300),
        'classifier__max_depth': stats.randint(3, 6),
        'classifier__learning_rate': stats.uniform(0.01, 0.3),
    }

    random_search = RandomizedSearchCV(pipe, param_distributions=param_dist, 
                                       scoring='accuracy', cv=cv, n_iter=100, 
                                       n_jobs=-1, random_state=42, refit=False)

    random_search.fit(X_train, y_train)
    random_search.best_params_
    
```

**Output:**
```
    {'classifier__learning_rate': 0.1079622306417506,
     'classifier__max_depth': 4,
     'classifier__n_estimators': 289}
     
```

In [ ]:
# Tuned XGBClassifier
xgb = XGBClassifier(objective='multi:softmax',
                    learning_rate=0.1, n_estimators=290, max_depth=4,
                    n_jobs=-1, random_state=42, use_label_encoder=False, eval_metric='mlogloss')

xgb_pipe = Pipeline(steps=[
    ('encoder', encoder),
    ('classifier', xgb)
])
xgb_pipe.fit(X_train, y_train)

In [ ]:
# Compute metrics for comparison for XGBClassifier
compute_comparison_metrics('XGBoost', xgb_pipe)
results

<a name='MLP'></a>
### 5.3.7 Multi-Layer Perceptron (Neural Network)

Starting with a simple MLP with 1 Input, 2 Hidden, and 1 Output Layers. The number of neurons per hidden is considered as half of that in the previous neuron, i.e. Hidden Layer-1 has 128 and Hidden Layer-2 has 64 (\~128/2) neurons.

Moreover, after testing with various activation functions, including Sigmoid and Tanh, Rectified Linear Unit (ReLU) for the hidden layers gave the most promising results.

$$
\text{ReLU}(z) = \max(0, z)
$$

Scikit-learn automatically assigns the activation function of the output layer as Softmax for a multi-class classification problem.

$$
\sigma(z)_i = \frac{e^{z_i}}{\sum_{k=1}^{K}{e}^{z_k}}
$$

The solver is chosen as 'Adam', which genrally works well on large datasets in terms of training time and validation score. Moreover, early_stopping=True has also been enabled with validation_fraction=0.1 so that model is able to generalize well and prevent overfitting.

**Code:**
```python
    # Tuning Neural Network for regularization strength
    model = MLPClassifier(hidden_layer_sizes=(128, 64), solver='adam',
                        early_stopping=True, validation_fraction=0.1, random_state=42)

    alpha_range = np.logspace(-4, 1, 6)

    pipe = Pipeline(steps=[
        ('encoder', encoder),
        ('classifier', model)
    ])

    train_scores, test_scores = multi_metric_validation_curve(pipe, X_train, y_train,
                                                              param_name='alpha', param_range=alpha_range,
                                                              metrics=['neg_log_loss', 'roc_auc_ovr_weighted'],
                                                              cv=cv)
    # Plot validation curve for Cross-Entropy Loss
    plot_validation_curve(-train_scores['neg_log_loss'], -test_scores['neg_log_loss'],
                          'MLP', 'alpha', alpha_range,
                          'Cross-Entropy Loss', logx=True)

```

**Output:**

<img src='https://drive.google.com/thumbnail?id=1BSlf-0odNdFAQBV7wtT61W-R9BrM-3i8&sz=w1000' width='1000'>

---

**Code:**
```python
    # Plot validation curve for ROC AUC
    plot_validation_curve(train_scores['roc_auc_ovr_weighted'], test_scores['roc_auc_ovr_weighted'],
                          'MLP', 'alpha', alpha_range,
                          'ROC AUC OVR Weighted', logx=True)
```

**Output:**

<img src='https://drive.google.com/thumbnail?id=1S1k9GSdbcIhCtHXvTRwm6xiMTeTvWK0i&sz=w1000' width='1000'>

For the Cross-Entropy Loss plot, minimum test loss occurs for alpha=0.1; however, the training loss increases around this range. ROC AUC is also roughly constant till alpha=1 (decreasing sharply from alpha=1).

Interestingly, for alpha=10, both train and test losses increase sharply, while ROC AUC values decrease sharply. This suggests underfitting of the MLP.

Therefore, alpha=0.1 is considered ahead as it minimizes the test loss.

In [ ]:
# Tuned Neural Network
mlp = MLPClassifier(hidden_layer_sizes=(128, 64),
                    solver='adam', alpha=0.1,
                    early_stopping=True, validation_fraction=0.1,
                    random_state=42)

mlp_pipe = Pipeline(steps=[
    ('encoder', encoder),
    ('classifier', mlp)
])
mlp_pipe.fit(X_train, y_train)

In [ ]:
# Compute metrics for comparison for MLP
compute_comparison_metrics('MLP', mlp_pipe)
results

In [ ]:
# Model Inspection
mlpc = mlp_pipe.named_steps['classifier']
print('Number of layers:', mlpc.n_layers_)
print('Number of features in:', mlpc.n_features_in_)
print('Number of outputs:', mlpc.n_outputs_)
print(f'Hidden Layer Activation: {mlpc.get_params()["activation"].title()}')
print(f'Output Layer Activation: {mlpc.out_activation_.title()}')

<a name='Model-Compare'></a>
## 5.4 Model Comparison

<a name='Compare-Metrics'></a>
### 5.4.1 Comparing Metrics

In [ ]:
# Display the Comparison Metrics
results

In [ ]:
# Compare Accuracies across Models
model_names = results['Model']

train_accuracies = [acc[0] for acc in results['Accuracy']]
test_accuracies = [acc[1] for acc in results['Accuracy']]

window = (0.9, 1.0)

plt.figure(figsize=(7, 7))
[plt.scatter(test_accuracies[i], train_accuracies[i]) for i in range(results.shape[0])]
plt.plot(window, window, color='black', linestyle='-')
plt.xlabel('Test Accuracy')
plt.ylabel('Train Accuracy')
plt.title('Train vs Test Accuracy')
plt.legend(results.Model.to_list())
plt.xlim(window)
plt.ylim(window)
plt.show()

As evident, all classifiers are scoring better than a DummyClassifier that predicts the most_frequent class (~0.58 Accuracy).

* XGBoost
   * Highest overall performance
   * Best balance between train and test accuracy
   * Suggests good generalization and predictive power

* Random Forest
   * Second-best performer
   * Good balance between train and test accuracy
   * Robust Classifier

* Logistic Regression
   * Simple yet effective
   * Very close performance to more complex models
   * Good generalization (train and test accuracies are close)

* MLP (Multi-Layer Perceptron)
   * Performs similarly to Logistic Regression
   * Captures non-linear relationships
   * Good balance between simplicity and performance

* SVC (Support Vector Classifier)
   * Solid performance, close to Logistic Regression and MLP
   * Good generalization ability due to its effectiveness in high-dimensional spaces

* K Nearest Neighbors
   * Shows signs of overfitting (high train accuracy, lower test accuracy)

* Decision Tree
   * Lowest overall accuracy among all models but still better than KNN on test set.


In [ ]:
# Compare Weighted Precisions, Weighted F1 Scores, and Weighted ROC AUC across Models
# Weighted Recall is same as Accuracy, hence not plotted

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))

train_precisions = [pre[0] for pre in results['Precision Weighted']]
test_precisions = [pre[1] for pre in results['Precision Weighted']]

train_f1 = [f1[0] for f1 in results['F1 Weighted']]
test_f1 = [f1[1] for f1 in results['F1 Weighted']]

train_roc_auc = [roc_auc[0] for roc_auc in results['ROC AUC OVR Weighted']]
test_roc_auc = [roc_auc[1] for roc_auc in results['ROC AUC OVR Weighted']]

[ax[0].scatter(test_precisions[i], train_precisions[i]) for i in range(results.shape[0])]
[ax[1].scatter(test_f1[i], train_f1[i]) for i in range(results.shape[0])]
[ax[2].scatter(test_roc_auc[i], train_roc_auc[i]) for i in range(results.shape[0])]

[ax[i].plot((0, 1), (0, 1), color='black', linestyle='-') for i in range(3)]

[[ax[i].set_xlabel(f'Test {metric}'),
  ax[i].set_ylabel(f'Train {metric}'),
  ax[i].set_title(f'Train vs Test {metric}')] for (i, metric) in enumerate(['Weighted Precision', 'Weighted F1 Score', 'Weighted ROC AUC'])]

ax[0].set_xlim(0.9, 1), ax[0].set_ylim(0.9, 1)
ax[1].set_xlim(0.9, 1), ax[1].set_ylim(0.9, 1)
ax[2].set_xlim(0.96, 1), ax[2].set_ylim(0.96, 1)

ax[1].legend(list(model_names), loc="best")
plt.show()


### Observations:
- **Overfitting**: K Nearest Neighbors (orange) appears to be the most overfitting model among the ones tested.
- **Robust Models**: Logistic Regression, Random Forest, and XGBoost are the most consistent across all metrics, indicating they are well-tuned and generalize well to the test set.
- **Overall Performance**: All models exhibit high performance, with most achieving close to perfect scores in terms of ROC AUC, indicating they are all effective for this classification task.



<a name='ROC-Curve'></a>
### 5.4.2 ROC Curves

In [ ]:
# Compiling all trained models
models = {
    'Logistic Regression': lr_pipe,
    'K Nearest Neighbors': knn_pipe,
    'Support Vector Classifier': svc_pipe,
    'Decision Tree': dt_pipe,
    'Random Forest': rf_pipe,
    'XGBoost': xgb_pipe,
    'Multi-Layer Perceptron': mlp_pipe
}

In [ ]:
classes = label_encoder.classes_
n_classes = len(classes)

fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(26, 17), sharex=True, sharey=True)

ax = ax.flatten()

# Iterate over each model
for model_name, model in models.items():
    y_val_pred_proba = model.predict_proba(X_val)
    
    for i in range(n_classes):
        y_val_binary = (y_val == i).astype(int)
        fpr, tpr, _ = roc_curve(y_val_binary, y_val_pred_proba[:, i])
        roc_auc = auc(fpr, tpr)
        ax[i].plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')

# Set common properties for all subplots
for i in range(n_classes):
    ax[i].plot([0, 1], [0, 1], 'k--')
    ax[i].set_xlabel('False Positive Rate')
    ax[i].set_ylabel('True Positive Rate')
    ax[i].set_title(f'ROC curve for Class {classes[i]}')
    ax[i].set_xlim([0, 1])
    ax[i].set_ylim([0, 1])
    ax[i].legend(loc='best')
    ax[i].grid(True)

plt.show()

The ROC Curves also show that XGBoost, Random Forest, and MLP are the best performing across the classes, while KNN and Decision Trees perform the worst. For classes Fraud and White Collar, Property, and Violent Crimes, almost all classifiers perform similarly.

**Remarks:**
- Given the diversity in performance metrics for the models, perhaps an **Ensemble** of these models can be trained and tested as well.

<a name='Ensemble'></a>
## 5.5 Ensemble

Ensemble Learning is a method to combine multiple models to produced improved results. While **Bagging** and **Boosting**, modelled before, only use a single type of model (in this case, a Decision Tree), methods like **Voting** and **Stacking** take advantage of wide variety of models.

<a name='Voting'></a>
### 5.5.1 Voting Classifier

In [ ]:
estimators = [('lr', lr_pipe),
              ('knn', knn_pipe),
              ('svc', svc_pipe),
              ('dt', dt_pipe),
              ('rf', rf_pipe),
              ('xgb', xgb_pipe),
              ('mlp', mlp_pipe)]

<a name='Hard-Voting'></a>
#### 5.5.1.1 Hard Voting Classifier
Final prediction is based on majority rule voting of predicted classes by the base estimators.

In [ ]:
hard_voting_model = VotingClassifier(
    estimators=estimators,
    voting='hard',
    n_jobs=-1
)
hard_voting_model.fit(X_train, y_train)

In [ ]:
# Compute metrics for comparison for Hard-Voting Model
print(f'Train Accuracy (Hard Voting)     : {hard_voting_model.score(X_train, y_train)}')
print(f'Validation Accuracy (Hard Voting): {hard_voting_model.score(X_val, y_val)}')

In [ ]:
# Classification Report for Hard Voting Model
y_val_pred = hard_voting_model.predict(X_val)
print(classification_report(y_val, y_val_pred))

<a name='Soft-Voting'></a>
#### 5.5.1.2 Soft Voting Classifier

Final Prediction is the argmax of the sum of prediction probabilities of the base estimators.

In [ ]:
soft_voting_model = VotingClassifier(
    estimators=estimators,
    voting='soft',
    n_jobs=-1
)
soft_voting_model.fit(X_train, y_train)

In [ ]:
cm = confusion_matrix(y_val, soft_voting_model.predict(X_val))
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.ylabel('True', fontsize=16)
plt.title('Predicted', fontsize=16)
plt.show()

In [ ]:
# Compute metrics for comparison for Soft-Voting Model
compute_comparison_metrics('Soft-Voting', soft_voting_model)
results

Both the variations of Voting Classifiers portrayed similar results and trained in similar duration of time.

Even though **Hard-Voting Model** has a slightly better Validation Accuracy, the **Soft-Voting Model** is the one to proceed with as it is known to be robust to outliers and can mitigate the bias in an imbalanced dataset because it fundamentally considers probabilities to come to a prediction.

<a name='Stacking'></a>
### 5.5.2 Stacking Classifier

A Meta Estimator (in this case, Logistic Regression) makes a prediction based on the prediction probability outputs of each Base Estimator.

In [ ]:
stacking_encoder = ColumnTransformer(
    transformers=[
        ('mlb', ModusOperandiBinarizer(), 'Modus_Operandi'),
        ('ohe', OneHotEncoder(handle_unknown='ignore'), ['Premise_Code', 'Area_ID', 'Location_Type', 'Victim_Sex', 'Victim_Descent', 'Weapon_Used_Code', 'Status'])  
    ],
    remainder='passthrough'
)

# KNN
best_params_knn = {'metric': 'manhattan', 'n_neighbors': 24, 'weights': 'distance'}
stacking_knn = KNeighborsClassifier(**best_params_knn, n_jobs=-1)
stacking_knn_pipe = Pipeline(steps=[
    ('encoder', stacking_encoder),
    ('feature_selection', SelectKBest(score_func=f_classif, k=30)),
    ('scaler', StandardScaler()),
    ('classifier', stacking_knn)
])

# MLP
stacking_mlp = MLPClassifier(hidden_layer_sizes=(128, 64), solver='adam', alpha=0.1, early_stopping=True, validation_fraction=0.1, random_state=42)
stacking_mlp_pipe = Pipeline(steps=[
    ('encoder', stacking_encoder),
    ('classifier', stacking_mlp)
])

# XGBoost
stacking_xgb = XGBClassifier(objective='multi:softmax', learning_rate=0.1, n_estimators=290, max_depth=4, n_jobs=-1, random_state=42, use_label_encoder=False, eval_metric='mlogloss')
stacking_xgb_pipe = Pipeline(steps=[
    ('encoder', stacking_encoder),
    ('classifier', stacking_xgb)
])

# Random Forest
stacking_rf = RandomForestClassifier(n_estimators=100, max_depth=16, max_features=None, bootstrap=True, random_state=42)
stacking_rf_pipe = Pipeline(steps=[
    ('encoder', stacking_encoder),  
    ('classifier', stacking_rf)
])

# Decision Tree
stacking_dt = DecisionTreeClassifier(ccp_alpha=0.0005, max_depth=20, min_samples_leaf=2, min_samples_split=2, random_state=42)
stacking_dt_pipe = Pipeline(steps=[
    ('encoder', stacking_encoder),
    ('classifier', stacking_dt)
])

# SVC
best_params_svc = {'C': 0.1, 'kernel': 'linear'}
stacking_svc = SVC(**best_params_svc, probability=True, random_state=42)
stacking_svc_pipe = Pipeline(steps=[
    ('encoder', stacking_encoder),
    ('classifier', stacking_svc)
])

# Logistic Regression
stacking_lr = LogisticRegression(C=1, max_iter=150, random_state=42, n_jobs=-1, solver='saga')
stacking_lr_pipe = Pipeline(steps=[
    ('encoder', stacking_encoder),
    ('scaler', StandardScaler()),
    ('classifier', stacking_lr)
])

# Stacking Classifier
stacking_clf = StackingClassifier(
    estimators=[
        ('knn', stacking_knn_pipe),
        ('mlp', stacking_mlp_pipe),
        ('xgb', stacking_xgb_pipe),
        ('rf', stacking_rf_pipe),
        ('dt', stacking_dt_pipe),
        ('svc', stacking_svc_pipe),
        ('lr', stacking_lr_pipe)
    ],
    final_estimator=LogisticRegression(),
    cv=2,
    n_jobs=-1
)

stacking_clf.fit(X_train, y_train)

In [ ]:
# Compute metrics for comparison for Stacking Model
compute_comparison_metrics('Stacking', stacking_clf)
results

**Stacking Model** outperforms all other classifiers in each metric.

Thus, Ensemble Learning has provided us with better classifiers than their individual base estimators.

<a name='Model-Compare-2'></a>
## 5.6 Model Comparison Part-2

Comparing Ensemble classifiers with previously trained models

In [ ]:
# Compiling all trained models
models = {
    'Logistic Regression': lr_pipe,
    'K Nearest Neighbors': knn_pipe,
    'Support Vector Classifier': svc_pipe,
    'Decision Tree': dt_pipe,
    'Random Forest': rf_pipe,
    'XGBoost': xgb_pipe,
    'Multi-Layer Perceptron': mlp_pipe,
    'Soft-Voting': soft_voting_model,
    'Stacking': stacking_clf
}

In [ ]:
# Compare Weighted Precisions, Weighted F1 Scores, and Weighted ROC AUC across Models
# Weighted Recall is same as Accuracy, hence not plotted

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))

train_precisions = [pre[0] for pre in results['Precision Weighted']]
test_precisions = [pre[1] for pre in results['Precision Weighted']]

train_f1 = [f1[0] for f1 in results['F1 Weighted']]
test_f1 = [f1[1] for f1 in results['F1 Weighted']]

train_roc_auc = [roc_auc[0] for roc_auc in results['ROC AUC OVR Weighted']]
test_roc_auc = [roc_auc[1] for roc_auc in results['ROC AUC OVR Weighted']]

[ax[0].scatter(test_precisions[i], train_precisions[i]) for i in range(results.shape[0])]
[ax[1].scatter(test_f1[i], train_f1[i]) for i in range(results.shape[0])]
[ax[2].scatter(test_roc_auc[i], train_roc_auc[i]) for i in range(results.shape[0])]

[ax[i].plot((0, 1), (0, 1), color='black', linestyle='-') for i in range(3)]

[[ax[i].set_xlabel(f'Test {metric}'),
  ax[i].set_ylabel(f'Train {metric}'),
  ax[i].set_title(f'Train vs Test {metric}')] for (i, metric) in enumerate(['Weighted Precision', 'Weighted F1 Score', 'Weighted ROC AUC'])]

ax[0].set_xlim(0.92, 1), ax[0].set_ylim(0.92, 1)
ax[1].set_xlim(0.92, 1), ax[1].set_ylim(0.92, 1)
ax[2].set_xlim(0.97, 1), ax[2].set_ylim(0.97, 1)

ax[1].legend(results.Model.to_list(), loc="best")
plt.show()

<a name='ROC-Curve-2'></a>
### 5.6.1 ROC Curves

In [ ]:
classes = label_encoder.classes_
n_classes = len(classes)

fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(26, 17), sharex=True, sharey=True)

ax = ax.flatten()

# Iterate over each model
for model_name, model in models.items():
    y_val_pred_proba = model.predict_proba(X_val)
    
    for i in range(n_classes):
        y_val_binary = (y_val == i).astype(int)
        fpr, tpr, _ = roc_curve(y_val_binary, y_val_pred_proba[:, i])
        roc_auc = auc(fpr, tpr)
        ax[i].plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')

# Set common properties for all subplots
for i in range(n_classes):
    ax[i].plot([0, 1], [0, 1], 'k--')
    ax[i].set_xlabel('False Positive Rate')
    ax[i].set_ylabel('True Positive Rate')
    ax[i].set_title(f'ROC curve for Class {classes[i]}')
    ax[i].set_xlim([0, 1])
    ax[i].set_ylim([0, 1])
    ax[i].legend(loc='best')
    ax[i].grid(True)

plt.show()

<a name='Confusion-Error-Matrix'></a>
### 5.6.2 Confusion and Error Matrices

In [ ]:
# Computing Confusion Matrix for Stacking, XGBoost, and Random Forest (Top-3 Models)
y_val_pred = stacking_clf.predict(X_val)
conf_mat_stacking = confusion_matrix(y_val, y_val_pred)

y_val_pred = xgb_pipe.predict(X_val)
conf_mat_xgb = confusion_matrix(y_val, y_val_pred)

y_val_pred = rf_pipe.predict(X_val)
conf_mat_rf = confusion_matrix(y_val, y_val_pred)

In [ ]:
# Rendering the Confusion Matrix
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(22, 5))

ConfusionMatrixDisplay(conf_mat_stacking, display_labels=[f'Class {i}' for i in range(n_classes)]).plot(ax=ax[0])
ConfusionMatrixDisplay(conf_mat_xgb, display_labels=[f'Class {i}' for i in range(n_classes)]).plot(ax=ax[1])
ConfusionMatrixDisplay(conf_mat_rf, display_labels=[f'Class {i}' for i in range(n_classes)]).plot(ax=ax[2])

ax[0].set_title('Confusion Matrix for Stacking Model')
ax[1].set_title('Confusion Matrix for XGB Model')
ax[2].set_title('Confusion Matrix for Random Forest Model')

plt.show()

In [ ]:
class_mapping = list(label_encoder.classes_)

class_mapping = {class_mapping[i]: i for i in range(len(class_mapping))}
class_mapping

Confusion Matrix are not very informative because the class imbalance skews the color gradient scale. Rather, Error Matrices are plotted ahead to understand certain model behavior.

> **Note:** For the purposes of model comparison, in this notebook an Error Matrix is defined as a way to respresent the Confusion Matrix without the correct classifications, thus highlighting the errors in prediction.

In [ ]:
# Computing and Rendering Error Matrics for Stacking, XGBoost and Random Forest
error_mat_stacking = np.copy(conf_mat_stacking)
error_mat_xgb = np.copy(conf_mat_xgb)
error_mat_rf = np.copy(conf_mat_rf)
for i in range(n_classes):
    error_mat_stacking[i][i] = 0
    error_mat_xgb[i][i] = 0
    error_mat_rf[i][i] = 0

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(22, 5))

ConfusionMatrixDisplay(error_mat_stacking, display_labels=[f'Class {i}' for i in range(n_classes)]).plot(ax=ax[0])
ConfusionMatrixDisplay(error_mat_xgb, display_labels=[f'Class {i}' for i in range(n_classes)]).plot(ax=ax[1])
ConfusionMatrixDisplay(error_mat_rf, display_labels=[f'Class {i}' for i in range(n_classes)]).plot(ax=ax[2])

ax[0].set_title('Confusion Matrix for Stacking Model')
ax[1].set_title('Confusion Matrix for XGB Model')
ax[2].set_title('Confusion Matrix for Random Forest Model')

plt.show()


- 'Crimes against Public Order' and 'Other Crimes': These classes are consistently misclassified as 'Violent Crimes' across all models. This indicates an overlap in features that make these categories difficult for the models to distinguish from 'Violent Crimes'.

- 'Violent Crimes': This category is frequently predicted for other classes, especially 'Crimes against Public Order' and 'Other Crimes', suggesting that the features indicative of violent crimes might be overrepresented or not well-separated from these other categories.

<a name='Submission'></a>
# 6. Submission

In [ ]:
final_model = XGBClassifier(objective='multi:softmax',
                    learning_rate=0.3, n_estimators=100,
                    n_jobs=-1, random_state=42, use_label_encoder=False, eval_metric='mlogloss')

final_model_pipe = Pipeline(steps=[
    ('encoder', encoder),
    ('classifier', final_model)
])
final_model_pipe.fit(preprocessed_train_df, target_encoded)

In [ ]:
predictions = label_encoder.inverse_transform(final_model_pipe.predict(preprocessed_test_df))
submission_df = pd.DataFrame({
    'ID': range(1, len(predictions) + 1),
    'Crime_Category': predictions
})
submission_df.to_csv('submission.csv', index=False)